# 第七章 深度学习在文本序列中的应用（7.1-7.2） 

文本序列在实际生活中有非常多的重要应用，如自动化翻译、机器作诗、人机对答等。

## 7.1 词嵌入 

词嵌入是把一个维数是所有词的数量的高维空间嵌入到一个维数低得多的连续向量空间中，每个单词或词组是映射到实数域上的向量。

在文本序列的研究中，如何描述、刻画、捕捉词与词之间的相关性，是第一重要的问题。

- 如果两个词经常在一起出现，那它们的相关性很强
- 若果两个词语义上非常相似，那它们的相关性很强

- 语义相关性

语义相关性是指把语义上相似的词聚到一起。 自然语言的一个基础目标就是，把一个个抽象的词或句子映射到一个欧式空间中。

在数学上想要达到目标，就要建立一个映射关系，将词或者短句映射到带有距离的高维欧式空间中。这样的目标称为词嵌入（Word Embedding）。

词嵌入就是要通过大量文本数据的学习，找到每一个词汇与高维空间的映射关系，表示该词汇在抽象空间中的位置，即它的坐标。

- 数学表达

这里给出三个词，酒店、宾馆和旅店，只关心它们之间的相对距离。假设酒店、宾馆和旅店的位置分别为V1、V2和V3,对三者做相同大小和方向的平移，所有的相对距离保持不变。

因此，任给一组真实的文本数据，可以有无限多种空间坐标的设定方法。对同一组数据，使用不同的起点，每次的结果、具体地位置是各不相同的，但是它们之间的相对距离是稳定的。

- 理论原理

词嵌入的理论基础由托马斯·米克罗夫（Tomas Miklov）等人在2013年ICLR大会上提出的。

如果能基于一段文本的上下文，很好的预测中间的Y，那么在虚拟空间中表达的距离关系和真实世界中看到的文本逻辑应该在很大程度上是相似的、自恰的、不矛盾的。

所以位置$X_i$的确定，最简单直白的做法是做一个超高维的多分类逻辑回归。 多分类是因为因变量Y可以是不同的词语。

- 程序实现

In [1]:
import pandas as pd
data = pd.read_csv(r"D:\datasets\深度学习王汉生\第七章数据\Comment.csv")
data.head()

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"


- 中文分词

在大多数情况下，对于分词方法我们的理解是相同的，我们可以通过一个叫jieba的分词软件进行中文分词。

In [2]:
import jieba
train_data = []
for line in data.review:
    line_fenci = jieba.lcut(line)
    train_data.append(line_fenci)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\leozhou\AppData\Local\Temp\jieba.cache
Loading model cost 1.038 seconds.
Prefix dict has been built successfully.


In [3]:
train_data[0]

['距离',
 '川沙',
 '公路',
 '较近',
 ',',
 '但是',
 '公交',
 '指示',
 '不',
 '对',
 ',',
 '如果',
 '是',
 '"',
 '蔡陆线',
 '"',
 '的话',
 ',',
 '会',
 '非常',
 '麻烦',
 '.',
 '建议',
 '用',
 '别的',
 '路线',
 '.',
 '房间',
 '较为简单',
 '.']

In [4]:
len(train_data)

7765

In [5]:
train_data[7764]

['说',
 '实在',
 '的',
 '我',
 '很',
 '失望',
 '，',
 '之前',
 '看',
 '了',
 '其他人',
 '的',
 '点评',
 '后',
 '觉得',
 '还',
 '可以',
 '才',
 '去',
 '的',
 '，',
 '结果',
 '让',
 '我们',
 '大跌眼镜',
 '。',
 '我',
 '想',
 '这家',
 '酒店',
 '以后',
 '无论如何',
 '我',
 '都',
 '不会',
 '再',
 '去',
 '了',
 '。']

- 从gensim.models载入Word2Vec

In [6]:
from gensim.models import Word2Vec                   #从gensim模块导入Word2Vec
model = Word2Vec(train_data,size=100,min_count=1)

In [7]:
model.wv['酒店']

array([-1.0282228 ,  1.2324519 ,  0.17820315, -0.53252655,  1.2772806 ,
        0.8106486 , -0.3053062 , -0.31731185,  1.4333003 , -0.7021956 ,
        0.14301397,  0.11553412,  0.40719816, -0.74316484, -0.9406167 ,
        1.3092499 , -1.2700808 ,  1.2666212 , -0.70316994, -1.2328326 ,
        0.07087798,  1.491957  , -0.8100923 ,  1.6636437 ,  0.6821522 ,
        1.1301208 ,  0.76852775,  1.112089  , -0.3284992 ,  0.43713787,
        0.11525412, -0.82373494, -0.29683304, -1.2188417 , -0.02695899,
       -0.59207934, -0.10653202,  0.2941139 , -0.15623155, -0.25648776,
       -0.3994668 , -0.7347719 , -0.09508806,  1.2866955 , -1.2220874 ,
        0.11374445, -0.9059884 , -0.01005956,  0.21380098,  0.17993331,
        0.15070143, -1.4969168 ,  1.1542424 ,  0.29252014,  0.4192348 ,
       -0.08380911, -1.5833961 ,  0.06580742, -0.2984971 ,  1.2020601 ,
       -0.52656984,  1.0791506 , -0.2872394 ,  0.14133202,  0.06200192,
        0.42784488,  0.40937263,  0.8222475 , -0.7720268 ,  0.37

In [8]:
len(model.wv['酒店'])

100

- 词语相似性结果展示

通过**model.wv.similarity**函数可以计算各个词语之间的相似性

In [9]:
model.wv.similarity('酒店','饭店')

0.7978145

In [10]:
model.wv.similarity('酒店','不错')

0.5612445

通过**wv.similar_by_word**函数可以找到和它最接近的n个词语(topn=n)

（书上文本有错，代码没错）

In [11]:
print('与酒店最接近的十个词语：')
for key in model.wv.similar_by_word('酒店',topn=10):
    print(key)

与酒店最接近的三个词语：
('饭店', 0.7978144884109497)
('店', 0.7904263138771057)
('网络系统', 0.7756055593490601)
('大雪纷飞', 0.7645303010940552)
('新悦', 0.7592548131942749)
('它', 0.7525931596755981)
('首屈一指', 0.7500748634338379)
('农家饭', 0.7369409799575806)
('评价', 0.7366847991943359)
('糟', 0.7355024814605713)


In [12]:
print("与不错最接近的十个词语：")
for key in model.wv.similar_by_word('不错',topn=10):
    print(key)

与不错最接近的三个词语：
('行', 0.8761014342308044)
('马马虎虎', 0.8672124147415161)
('满好', 0.8663299083709717)
('比较满意', 0.8649245500564575)
('一般', 0.8645418882369995)
('挺不错', 0.8571749925613403)
('挺大', 0.8506811261177063)
('不怎么样', 0.8468936681747437)
('开裂', 0.8373464345932007)
('还好', 0.837342381477356)


In [13]:
help(model.wv.similar_by_word)

Help on method similar_by_word in module gensim.models.keyedvectors:

similar_by_word(word, topn=10, restrict_vocab=None) method of gensim.models.keyedvectors.Word2VecKeyedVectors instance
    Find the top-N most similar words.
    
    Parameters
    ----------
    word : str
        Word
    topn : int or None, optional
        Number of top-N similar words to return. If topn is None, similar_by_word returns
        the vector of similarity scores.
    restrict_vocab : int, optional
        Optional integer which limits the range of vectors which
        are searched for most-similar values. For example, restrict_vocab=10000 would
        only check the first 10000 word vectors in the vocabulary order. (This may be
        meaningful if you've sorted the vocabulary by descending frequency.)
    
    Returns
    -------
    list of (str, float) or numpy.array
        When `topn` is int, a sequence of (word, similarity) is returned.
        When `topn` is None, then similarities for al

In [14]:
list = model.wv.similar_by_word('不错',topn=None)
list

array([0.39754754, 0.4835    , 0.44730693, ..., 0.0130267 , 0.17133108,
       0.13979328], dtype=float32)

In [15]:
len(list)

29715

## 7.2 机器作诗初级：逻辑回归 

机器作诗其实就是一个回归分析的概率问题。写诗本质上就是在不自觉地做一个关于X和Y的回归分析，如果这个X能够机器准确地预测Y，就说明它对仗工整

这个问题中分类变量Y是不同的字，X是在Y之前的三个字，如果前面不够三个字，就补充一些毫无意义的字母符号，这就是逻辑回归的原理。

- 读入数据

In [16]:
import pandas as pd
poems_text = pd.read_table(r'D:\datasets\深度学习王汉生\第七章数据\poems_clean.txt',header = None)
poems_text.columns = ['text']
poems_text.head()

,text
0,首春:寒随穷律变 春逐鸟声开 初风飘带柳 晚雪间花梅 碧林青旧竹 绿沼翠新苔 芝田初雁去 绮...
1,初晴落景:晚霞聊自怡 初晴弥可喜 日晃百花色 风动千林翠 池鱼跃不同 园鸟声还异 寄言博通者...
2,度秋:夏律昨留灰 秋箭今移晷 峨嵋岫初出 洞庭波渐起 桂白发幽岩 菊黄开灞涘 运流方可叹 含...
3,仪鸾殿早秋:寒惊蓟门叶 秋发小山枝 松阴背日转 竹影避风移 提壶菊花岸 高兴芙蓉池 欲知凉气...
4,山阁晚秋:山亭秋色满 岩牖凉风度 疏兰尚染烟 残菊犹承露 古石衣新苔 新巢封古树 历览情无极...


In [17]:
poems_text.shape

(24117, 1)

In [18]:
poems_text['text'][3864]

'静夜思:床前明月光 疑是地上霜 举头望明月 低头思故乡  '

- 数据预处理

In [19]:
import string
import numpy as np
poems_new = []
for line in poems_text['text']:  
     title, poem = line.split(':')
     poem = poem.replace(' ', '') 
     poem = 'bbb' + poem
     poems_new.append(poem)

代码改错：最后一行 poems_new.append(poem) 删去list 否则显示列表不可调用

In [20]:
poems_new

['bbb寒随穷律变春逐鸟声开初风飘带柳晚雪间花梅碧林青旧竹绿沼翠新苔芝田初雁去绮树巧莺来',
 'bbb晚霞聊自怡初晴弥可喜日晃百花色风动千林翠池鱼跃不同园鸟声还异寄言博通者知予物外志',
 'bbb夏律昨留灰秋箭今移晷峨嵋岫初出洞庭波渐起桂白发幽岩菊黄开灞涘运流方可叹含毫属微理',
 'bbb寒惊蓟门叶秋发小山枝松阴背日转竹影避风移提壶菊花岸高兴芙蓉池欲知凉气早巢空燕不窥',
 'bbb山亭秋色满岩牖凉风度疏兰尚染烟残菊犹承露古石衣新苔新巢封古树历览情无极咫尺轮光暮',
 'bbb翠野驻戎轩卢龙转征旆遥山丽如绮长流萦似带海气百重楼岩松千丈盖兹焉可游赏何必襄城外',
 'bbb春蒐驰骏骨总辔俯长河霞处流萦锦风前漾卷罗水花翻照树堤兰倒插波岂必汾阴曲秋云发棹歌',
 'bbb重峦俯渭水碧嶂插遥天出红扶岭日入翠贮岩烟叠松朝若夜复岫阙疑全对此恬千虑无劳访九仙',
 'bbb萧条起关塞摇飏下蓬瀛拂林花乱彩响谷鸟分声披云罗影散泛水织文生劳歌大风曲威加四海清',
 'bbb罩云飘远岫喷雨泛长河低飞昏岭腹斜足洒岩阿泫丛珠缔叶起溜镜图波濛柳添丝密含吹织空罗',
 'bbb洁野凝晨曜装墀带夕晖集条分树玉拂浪影泉玑色洒妆台粉花飘绮席衣入扇萦离匣点素皎残机',
 'bbb红轮不暂驻乌飞岂复停岑霞渐渐落溪阴寸寸生藿叶随光转葵心逐照倾晚烟含树色栖鸟杂流声',
 'bbb高轩临碧渚飞檐迥架空余花攒镂槛残柳散雕栊岸菊初含蕊园梨始带红莫虑昆山暗还共尽杯中',
 'bbb华林满芳景洛阳遍阳春朱颜含远日翠色影长津乔柯啭娇鸟低枝映美人昔作园中实今来席上珍',
 'bbb玉衡流桂圃成蹊正可寻莺啼密叶外蝶戏脆花心丽景光朝彩轻霞散夕阴暂顾晖章侧还眺灵山林',
 'bbb岸曲非千里桥斜异七星暂低逢辇度还高值浪惊水摇文鹢动缆转锦花萦远近随轮影轻重应人行',
 'bbb拂霞疑电落腾虚状写虹屈伸烟雾里低举白云中纷披乍依迥掣曳或随风念兹轻薄质无翅强摇空',
 'bbb凿门初奉律仗战始临戎振鳞方跃浪骋翼正凌风未展六奇术先亏一篑功防身岂乏智殉命有余忠',
 'bbb晦魄移中律凝暄起丽城罩云朝盖上穿露晓珠呈笑树花分色啼枝鸟合声披襟欢眺望极目畅春情',
 'bbb秋日凝翠岭凉吹肃离宫荷疏一盖缺树冷半帷空侧阵移鸿影圆花钉菊丛摅怀俗尘外高眺白云中',
 'bbb斜廊连绮阁初月照宵帏塞冷鸿飞疾园秋蝉噪迟露结林疏叶寒轻菊吐滋愁心逢此节

In [21]:
XY = []
for poem in poems_new:
    for i in range(len(poem)-3):
        x1 = poem[i]
        x2 = poem[i+1]
        x3 = poem[i+2]       
        y = poem[i+3]
        XY.append([x1,x2,x3,y])
print("原始诗句：")
print(poems_text['text'][3864])
print("\n 训练数据")
print(["X1","X2","X3","X4"])
for i in range(132763,132773):
    print(XY[i])

原始诗句：
静夜思:床前明月光 疑是地上霜 举头望明月 低头思故乡  

 训练数据
['X1', 'X2', 'X3', 'X4']
['b', 'b', 'b', '床']
['b', 'b', '床', '前']
['b', '床', '前', '明']
['床', '前', '明', '月']
['前', '明', '月', '光']
['明', '月', '光', '疑']
['月', '光', '疑', '是']
['光', '疑', '是', '地']
['疑', '是', '地', '上']
['是', '地', '上', '霜']


- 从字符到数字的映射字典

Tensorflow 不能处理非数字型的向量或矩阵，所以需要把每一个涉及到的汉字或者英文字符用一个整数token代替。

In [22]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(XY)
print(tokenizer.word_index)

vocab_size = len(tokenizer.word_index) + 1
vocab_size

{'b': 1, '不': 2, '人': 3, '山': 4, '风': 5, '日': 6, '无': 7, '一': 8, '云': 9, '春': 10, '花': 11, '月': 12, '何': 13, '水': 14, '来': 15, '上': 16, '有': 17, '时': 18, '秋': 19, '中': 20, '天': 21, '年': 22, '归': 23, '夜': 24, '江': 25, '相': 26, '去': 27, '知': 28, '长': 29, '君': 30, '见': 31, '此': 32, '白': 33, '心': 34, '客': 35, '自': 36, '行': 37, '处': 38, '生': 39, '为': 40, '里': 41, '寒': 42, '空': 43, '在': 44, '雨': 45, '下': 46, '是': 47, '清': 48, '落': 49, '得': 50, '高': 51, '如': 52, '远': 53, '多': 54, '路': 55, '明': 56, '门': 57, '未': 58, '青': 59, '别': 60, '南': 61, '树': 62, '尽': 63, '今': 64, '家': 65, '声': 66, '事': 67, '应': 68, '城': 69, '草': 70, '入': 71, '千': 72, '深': 73, '色': 74, '出': 75, '雪': 76, '新': 77, '独': 78, '向': 79, '还': 80, '思': 81, '前': 82, '闲': 83, '道': 84, '流': 85, '开': 86, '三': 87, '烟': 88, '酒': 89, '更': 90, '万': 91, '西': 92, '朝': 93, '欲': 94, '谁': 95, '看': 96, '东': 97, '子': 98, '飞': 99, '回': 100, '愁': 101, '玉': 102, '与': 103, '望': 104, '满': 105, '林': 106, '闻': 107, '马': 108, '到': 109, '已': 110, '地': 11

5547

代码改错： 第三行tokenizer.fit_on_texts(XY) poems_new改成XY

- 数字矩阵

In [23]:
XY_digit = np.array(tokenizer.texts_to_sequences(XY))
X_digit = XY_digit[:, :3]
Y_digit = XY_digit[:,3]
for i in range(132763,132773):
    print("{:<35}".format(str(XY[i])),'\t',"{:<30}".format(str(X_digit[i])),"\t",Y_digit[i])

['b', 'b', 'b', '床']                	 [1 1 1]                        	 560
['b', 'b', '床', '前']                	 [  1   1 560]                  	 82
['b', '床', '前', '明']                	 [  1 560  82]                  	 56
['床', '前', '明', '月']                	 [560  82  56]                  	 12
['前', '明', '月', '光']                	 [82 56 12]                     	 140
['明', '月', '光', '疑']                	 [ 56  12 140]                  	 429
['月', '光', '疑', '是']                	 [ 12 140 429]                  	 47
['光', '疑', '是', '地']                	 [140 429  47]                  	 111
['疑', '是', '地', '上']                	 [429  47 111]                  	 16
['是', '地', '上', '霜']                	 [ 47 111  16]                  	 202


代码改错： 最后一行 删去(X_digit[i])前面的list

### 7.2.3 原理实现：逻辑回归 

用逻辑回归解决汉字预测问题，词嵌入是必须要做的，词嵌入能大大降低模型中的参数。

词嵌入的两种方案：
- 用一个独立的文本学习每个词在虚拟空间中的相对位置，然后将学习出来的位置坐标作为 变量直接嵌套在模型里。——通用性强
- 把词嵌入直接嵌入后面的逻辑回归模型中，然后让模型和算法按照一定的标准自动寻找最好的嵌入参数和位置表达。——写诗这个任务更佳

In [24]:
from keras.layers import Input, Embedding
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Activation, Embedding, Flatten
hidden_size = 256

inp = Input(shape=(3,))
x = Embedding(vocab_size, hidden_size)(inp)
x = Flatten()(x)
x = Dense(vocab_size)(x)
pred = Activation('softmax')(x)

model = Model(inp, pred)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
embedding (Embedding)        (None, 3, 256)            1420032   
_________________________________________________________________
flatten (Flatten)            (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 5547)              4265643   
_________________________________________________________________
activation (Activation)      (None, 5547)              0         
Total params: 5,685,675
Trainable params: 5,685,675
Non-trainable params: 0
_________________________________________________________________


- 模型编译与拟合

In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X_digit,Y_digit,test_size=0.2,random_state=0)

from keras.optimizers import Adam
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(lr=0.001))
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=10000,epochs=10)

Epoch 1/10
65/65 [==============================] - 7s 112ms/step - loss: 8.0991 - val_loss: 7.0638
Epoch 2/10
65/65 [==============================] - 7s 107ms/step - loss: 6.9650 - val_loss: 6.9237
Epoch 3/10
65/65 [==============================] - 7s 106ms/step - loss: 6.8240 - val_loss: 6.7710
Epoch 4/10
65/65 [==============================] - 7s 107ms/step - loss: 6.6266 - val_loss: 6.5936
Epoch 5/10
65/65 [==============================] - 7s 107ms/step - loss: 6.4188 - val_loss: 6.4398
Epoch 6/10
65/65 [==============================] - 7s 108ms/step - loss: 6.2309 - val_loss: 6.3199
Epoch 7/10
65/65 [==============================] - 7s 108ms/step - loss: 6.0646 - val_loss: 6.2274
Epoch 8/10
65/65 [==============================] - 7s 108ms/step - loss: 5.9140 - val_loss: 6.1563
Epoch 9/10
65/65 [==============================] - 7s 108ms/step - loss: 5.7763 - val_loss: 6.1027
Epoch 10/10
65/65 [==============================] - 7s 109ms/step - loss: 5.6491 - val_loss: 6.0635

- 预测结果

In [26]:
sample_text = ['床', '前', '明'] 
print(sample_text)
sample_index = tokenizer.texts_to_sequences(sample_text)
print(sample_index)
word_prob = model.predict(np.array(sample_index).reshape(1, 3))[0]
print(tokenizer.index_word[word_prob.argmax()], word_prob.max())

['床', '前', '明']
[[560], [82], [56]]
月 0.34873238


### 使用逻辑回归写藏头诗 

In [33]:
poem_incomplete = 'bbb熊****大****很****帅****'
poem_index = []
poem_text = ''
for i in range(len(poem_incomplete)):
    current_word = poem_incomplete[i]
    
    if current_word != '*':
        index = tokenizer.word_index[current_word]
    else:
        #根据前三个词预测
        x = poem_index[-3:]
        y = model.predict(np.expand_dims(x,axis=0))[0]
        index = y.argmax()
        current_word = tokenizer.index_word[index]
        
    poem_index.append(index)
    poem_text  = poem_text + current_word
    
poem_text = poem_text[3:]
print(poem_text[0:5])
print(poem_text[5:10])
print(poem_text[10:15])
print(poem_text[15:20])

熊马蹄边草
大堤春风吹
很幕下山河
帅有情人不


## 7.3 机器作诗进阶1：RNN 